# Helsinki Property Visualization

This notebook visualizes property data for Helsinki, using data from the National Land Survey of Finland. It demonstrates how to load, filter, and plot geospatial data with a map background.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import contextily as cx
import fiona
from IPython.display import display, Markdown

# --- Configuration ---
gpkg_path = os.path.join('..', 'data', 'open', 'kiinteistorekisterikartta.gpkg')
municipality_zip_path = os.path.join('..', 'data', 'open', 'TietoaKuntajaosta_2025_10k.zip')
helsinki_municipality_name = 'Helsinki'

file_exists = os.path.exists(gpkg_path) and os.path.exists(municipality_zip_path)

if not file_exists:
    display(Markdown('## ❌ Data Files Not Found\nPlease ensure both `kiinteistorekisterikartta.gpkg` and `TietoaKuntajaosta_2025_10k.zip` are in the `data/open` directory.'))
else:
    display(Markdown('## ✅ All Data Files Found'))

## 1. Load and Filter Helsinki Data

In [ ]:
def get_helsinki_boundary():
    with fiona.open(f'zip://{municipality_zip_path}') as collection:
        municipalities = gpd.GeoDataFrame.from_features(collection, crs=collection.crs)
    helsinki = municipalities[municipalities['namefin'] == helsinki_municipality_name]
    return helsinki.geometry.union_all()

if file_exists:
    try:
        helsinki_boundary = get_helsinki_boundary()
        
        # Load the two layers of interest
        gdf_kiinteisto = gpd.read_file(gpkg_path, layer='KiinteistorajanSijaintitiedot')
        gdf_palsta = gpd.read_file(gpkg_path, layer='PalstanSijaintitiedot')
        
        # Filter for Helsinki
        helsinki_kiinteisto = gdf_kiinteisto[gdf_kiinteisto.within(helsinki_boundary)]
        helsinki_palsta = gdf_palsta[gdf_palsta.within(helsinki_boundary)]
        
        display(Markdown(f'Loaded **{len(helsinki_kiinteisto)}** `KiinteistorajanSijaintitiedot` features for Helsinki.'))
        display(Markdown(f'Loaded **{len(helsinki_palsta)}** `PalstanSijaintitiedot` features for Helsinki.'))
        
    except Exception as e:
        display(Markdown(f'### Error loading data: {e}'))

## 2. Visualize Two Random Properties

In [ ]:
def plot_property(gdf, title):
    property_to_plot = gdf.sample(n=1, random_state=None)
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    property_to_plot.to_crs(epsg=3857).plot(ax=ax, alpha=0.5, edgecolor='k')
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
    ax.set_title(title)
    ax.set_axis_off()
    plt.show()

if 'helsinki_kiinteisto' in locals() and not helsinki_kiinteisto.empty:
    plot_property(helsinki_kiinteisto, 'Random Property from KiinteistorajanSijaintitiedot')

if 'helsinki_palsta' in locals() and not helsinki_palsta.empty:
    plot_property(helsinki_palsta, 'Random Property from PalstanSijaintitiedot')

## 3. Visualize Properties within a Random Postal Code Area

In [ ]:
if 'helsinki_palsta' in locals() and not helsinki_palsta.empty:
    # This is a placeholder for actual postal code data
    # In a real scenario, you would load a postal code shapefile
    postal_codes_gdf = gpd.read_file(gpd.datasets.get_path('nybb')) # Using a sample dataset for now
    random_postal_code = postal_codes_gdf.sample(n=1, random_state=42)
    
    # Find properties within the random postal code area
    properties_in_postal_code = helsinki_palsta[helsinki_palsta.within(random_postal_code.union_all())]
    
    if not properties_in_postal_code.empty:
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        properties_in_postal_code.to_crs(epsg=3857).plot(ax=ax, alpha=0.7, edgecolor='blue', facecolor='lightblue')
        random_postal_code.to_crs(epsg=3857).plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
        cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
        ax.set_title(f'Properties within Postal Code: {random_postal_code.BoroName.iloc[0]}')
        ax.set_axis_off()
        plt.show()
    else:
        display(Markdown('No properties found in the selected random postal code area.'))

## ✅ Verification Cell

In [ ]:
display(Markdown('## ✅ Notebook Executed Successfully'))